# Ventilator Fine-tune Regression

I train as a classification task. (https://www.kaggle.com/takamichitoda/ventilator-train-classification?scriptVersionId=76597714)

The classification converges earlier than regression.

In my experience, to got  OOF score=0.17xx by using 4 layer bi-LSTM model
- regression needs 150 epoch
- classification needs 50 epoch

However, the classification case easily makes overfit.

<img src="https://raw.githubusercontent.com/trtd56/RFCX/main/tmp/%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%83%E3%83%88%202021-10-14%2015.40.55.png" width="400">

So in this code, I fine-tuning the regression task by using [classification model weight](https://www.kaggle.com/takamichitoda/ventilator-classification-model).

Classification model result is CV=0.1708 / LB=0.155, this model result is CV=0.1765 / LB=0.165.

If I use custom header based on 1d-CNN (the implementation like [here](https://www.kaggle.com/takamichitoda/ventilator-1dcnn-lstm)), I got CV=0.1622 / LB=0.150.

## UPDATE

|Date|Version|Detail|CV|LB|
|--|--|--|--|--|
|2021.10.27|4|Transformer Header|0.1665|0.1545|

In [ ]:
!pip install -U torch wandb transformers

In [ ]:
from kaggle_secrets import UserSecretsClient
secret_label = "wandb"
secret_value = UserSecretsClient().get_secret(secret_label)
!wandb login $secret_value

In [ ]:
import gc
import os
import random
import wandb
import math

import numpy as np
import pandas as pd

from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import AdamW
from transformers import get_cosine_schedule_with_warmup
from sklearn.preprocessing import RobustScaler

device = torch.device("cuda")

In [ ]:
class config:
    EXP_NAME = "exp137_transformer_head"
    CLS_MODEL = "../input/ventilator-classification-model/exp075_dropout0"
    
    INPUT = "/kaggle/input/ventilator-pressure-prediction"
    OUTPUT = "/kaggle/working"
    N_FOLD = 5
    SEED = 0
    
    LR = 5e-3
    N_EPOCHS = 50
    EMBED_SIZE = 64
    HIDDEN_SIZE = 256
    BS = 512
    WEIGHT_DECAY = 1e-3

    USE_LAG = 4
    CATE_FEATURES = ['R_cate', 'C_cate', 'RC_dot', 'RC_sum']
    CONT_FEATURES = ['u_in', 'u_out', 'time_step'] + ['u_in_cumsum', 'u_in_cummean', 'area', 'cross', 'cross2']
    LAG_FEATURES = ['breath_time']
    LAG_FEATURES += [f'u_in_lag_{i}' for i in range(1, USE_LAG+1)]
    LAG_FEATURES += [f'u_in_time{i}' for i in range(1, USE_LAG+1)]
    LAG_FEATURES += [f'u_out_lag_{i}' for i in range(1, USE_LAG+1)]
    ALL_FEATURES = CATE_FEATURES + CONT_FEATURES + LAG_FEATURES
    NORM_FEATURES = CONT_FEATURES + LAG_FEATURES
    
    NOT_WATCH_PARAM = ['INPUT']

In [ ]:
def set_seed(seed=config.SEED):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
class VentilatorDataset(Dataset):
    
    def __init__(self, df, label_dic=None):
        self.dfs = [_df for _, _df in df.groupby("breath_id")]
        self.label_dic = label_dic
        
    def __len__(self):
        return len(self.dfs)
    
    def __getitem__(self, item):
        df = self.dfs[item]
        X = df[config.ALL_FEATURES].values
        y = df['pressure'].values

        d = {
            "X": torch.tensor(X).float(),
            "y" : torch.tensor(y).float(),
        }
        return d

In [ ]:
class VentilatorModel(nn.Module):
    
    def __init__(self):
        super(VentilatorModel, self).__init__()
        self.r_emb = nn.Embedding(3, 2, padding_idx=0)
        self.c_emb = nn.Embedding(3, 2, padding_idx=0)
        self.rc_dot_emb = nn.Embedding(8, 4, padding_idx=0)
        self.rc_sum_emb = nn.Embedding(8, 4, padding_idx=0)
        self.seq_emb = nn.Sequential(
            nn.Linear(12+len(config.NORM_FEATURES), config.EMBED_SIZE),
            nn.LayerNorm(config.EMBED_SIZE),
        )
        
        self.lstm = nn.LSTM(config.EMBED_SIZE, config.HIDDEN_SIZE, batch_first=True, bidirectional=True, num_layers=4, dropout=0.0)
        self.head = nn.Sequential(
            nn.Linear(config.HIDDEN_SIZE * 2, config.HIDDEN_SIZE * 2),
            nn.LayerNorm(config.HIDDEN_SIZE * 2),
            nn.ReLU(),
            nn.Linear(config.HIDDEN_SIZE * 2, 950),
        )

    def forward(self, X):
        # embed
        bs = X.shape[0]
        r_emb = self.r_emb(X[:,:,0].long()).view(bs, 80, -1)
        c_emb = self.c_emb(X[:,:,1].long()).view(bs, 80, -1)
        rc_dot_emb = self.rc_dot_emb(X[:,:,2].long()).view(bs, 80, -1)
        rc_sum_emb = self.rc_sum_emb(X[:,:,3].long()).view(bs, 80, -1)
        
        seq_x = torch.cat((r_emb, c_emb, rc_dot_emb, rc_sum_emb, X[:, :, 4:]), 2)
        emb_x = self.seq_emb(seq_x)
        
        out, _ = self.lstm(emb_x, None)

        return out


class VentilatorModelRegr(nn.Module):
    
    def __init__(self, load_path):
        super(VentilatorModelRegr, self).__init__()
        self.cls_model = VentilatorModel()
        self.cls_model.load_state_dict(torch.load(load_path))

        encoder_layers = nn.TransformerEncoderLayer(d_model=config.HIDDEN_SIZE * 2, nhead=1, dim_feedforward=2048, dropout=0.0, batch_first=True)
        self.regression = nn.Sequential(
            #nn.Linear(config.HIDDEN_SIZE * 2, config.HIDDEN_SIZE * 2),
            nn.TransformerEncoder(encoder_layers, num_layers=1),
            nn.LayerNorm(config.HIDDEN_SIZE * 2),
            nn.ReLU(),
            nn.Linear(config.HIDDEN_SIZE * 2, 1),
        )

    def forward(self, X, y=None):
        out = self.cls_model(X)
        regr = self.regression(out)

        if y is None:
            loss = None
        else:
            loss = self.loss_fn(regr.squeeze(2), y)
            
        return regr, loss
    
    def loss_fn(self, y_pred, y_true):
        criterion = nn.SmoothL1Loss()
        loss = criterion(y_pred, y_true)
        return loss

    def freeze_cls(self):
        for param in self.cls_model.parameters():
            param.requires_grad = False

In [ ]:
def train_loop(model, optimizer, scheduler, loader):
    losses, lrs = [], []
    model.train()
    optimizer.zero_grad()
    for d in loader:
        out, loss = model(d['X'].to(device), d['y'].to(device))
        
        losses.append(loss.item())
        step_lr = np.array([param_group["lr"] for param_group in optimizer.param_groups]).mean()
        lrs.append(step_lr)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()

    return np.array(losses).mean(), np.array(lrs).mean()

def valid_loop(model, loader, target_dic_inv):
    losses, predicts = [], []
    model.eval()
    for d in loader:
        with torch.no_grad():
            out, loss = model(d['X'].to(device), d['y'].to(device))
        losses.append(loss.item())
        predicts.append(out.cpu())
    return np.array(losses).mean(), torch.vstack(predicts).squeeze(2).numpy().reshape(-1)

def test_loop(model, loader, target_dic_inv):
    predicts = []
    model.eval()
    for d in loader:
        with torch.no_grad():
            out, _ = model(d['X'].to(device))
        predicts.append(out.cpu())
    return torch.vstack(predicts).squeeze(2).numpy().reshape(-1)

In [ ]:
def add_feature(df):
    df['time_delta'] = df.groupby('breath_id')['time_step'].diff().fillna(0)
    df['delta'] = df['time_delta'] * df['u_in']
    df['area'] = df.groupby('breath_id')['delta'].cumsum()

    df['cross']= df['u_in']*df['u_out']
    df['cross2']= df['time_step']*df['u_out']
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    df['one'] = 1
    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
    df['u_in_cummean'] =df['u_in_cumsum'] / df['count']
    
    df = df.drop(['count','one'], axis=1)
    return df

def add_lag_feature(df):
    # https://www.kaggle.com/kensit/improvement-base-on-tensor-bidirect-lstm-0-173
    for lag in range(1, config.USE_LAG+1):
        df[f'breath_id_lag{lag}']=df['breath_id'].shift(lag).fillna(0)
        df[f'breath_id_lag{lag}same']=np.select([df[f'breath_id_lag{lag}']==df['breath_id']], [1], 0)

        # u_in 
        df[f'u_in_lag_{lag}'] = df['u_in'].shift(lag).fillna(0) * df[f'breath_id_lag{lag}same']
        df[f'u_in_time{lag}'] = df['u_in'] - df[f'u_in_lag_{lag}']
        df[f'u_out_lag_{lag}'] = df['u_out'].shift(lag).fillna(0) * df[f'breath_id_lag{lag}same']

    # breath_time
    df['time_step_lag'] = df['time_step'].shift(1).fillna(0) * df[f'breath_id_lag{lag}same']
    df['breath_time'] = df['time_step'] - df['time_step_lag']

    drop_columns = ['time_step_lag']
    drop_columns += [f'breath_id_lag{i}' for i in range(1, config.USE_LAG+1)]
    drop_columns += [f'breath_id_lag{i}same' for i in range(1, config.USE_LAG+1)]
    df = df.drop(drop_columns, axis=1)

    # fill na by zero
    df = df.fillna(0)
    return df

c_dic = {10: 0, 20: 1, 50:2}
r_dic = {5: 0, 20: 1, 50:2}
rc_sum_dic = {v: i for i, v in enumerate([15, 25, 30, 40, 55, 60, 70, 100])}
rc_dot_dic = {v: i for i, v in enumerate([50, 100, 200, 250, 400, 500, 2500, 1000])}    

def add_category_features(df):
    df['C_cate'] = df['C'].map(c_dic)
    df['R_cate'] = df['R'].map(r_dic)
    df['RC_sum'] = (df['R'] + df['C']).map(rc_sum_dic)
    df['RC_dot'] = (df['R'] * df['C']).map(rc_dot_dic)
    return df

norm_features = config.CONT_FEATURES + config.LAG_FEATURES
def norm_scale(train_df, test_df):
    scaler = RobustScaler()
    all_u_in = np.vstack([train_df[norm_features].values, test_df[norm_features].values])
    scaler.fit(all_u_in)
    train_df[norm_features] = scaler.transform(train_df[norm_features].values)
    test_df[norm_features] = scaler.transform(test_df[norm_features].values)
    return train_df, test_df

In [ ]:
def main():
    
    train_df = pd.read_csv(f"{config.INPUT}/train.csv")
    test_df = pd.read_csv(f"{config.INPUT}/test.csv")
    sub_df = pd.read_csv(f"{config.INPUT}/sample_submission.csv")
    oof = np.zeros(len(train_df))
    test_preds_lst = []

    target_dic = {v:i for i, v in enumerate(sorted(train_df['pressure'].unique().tolist()))}
    target_dic_inv = {v: k for k, v in target_dic.items()}

    gkf = GroupKFold(n_splits=config.N_FOLD).split(train_df, train_df.pressure, groups=train_df.breath_id)
    for fold, (_, valid_idx) in enumerate(gkf):
        train_df.loc[valid_idx, 'fold'] = fold

    train_df = add_feature(train_df)
    test_df = add_feature(test_df)
    train_df = add_lag_feature(train_df)
    test_df = add_lag_feature(test_df)
    train_df = add_category_features(train_df)
    test_df = add_category_features(test_df)
    train_df, test_df = norm_scale(train_df, test_df)

    test_df['pressure'] = -1
    test_dset = VentilatorDataset(test_df)
    test_loader = DataLoader(test_dset, batch_size=config.BS,
                             pin_memory=True, shuffle=False, drop_last=False, num_workers=os.cpu_count())
    
    for fold in range(config.N_FOLD):
        print(f'Fold-{fold}')
        train_dset = VentilatorDataset(train_df.query(f"fold!={fold}"), target_dic)
        valid_dset = VentilatorDataset(train_df.query(f"fold=={fold}"), target_dic)

        set_seed()
        train_loader = DataLoader(train_dset, batch_size=config.BS,
                                  pin_memory=True, shuffle=True, drop_last=True, num_workers=os.cpu_count(),
                                  worker_init_fn=lambda x: set_seed())
        valid_loader = DataLoader(valid_dset, batch_size=config.BS,
                                  pin_memory=True, shuffle=False, drop_last=False, num_workers=os.cpu_count())

        load_path = f"{config.OUTPUT}/{config.CLS_MODEL}/ventilator_f{fold}_best_model.bin"
        model = VentilatorModelRegr(load_path)
        model.to(device)
        model.freeze_cls()

        optimizer = AdamW(model.parameters(), lr=config.LR, weight_decay=config.WEIGHT_DECAY)
        num_train_steps = int(len(train_loader) * config.N_EPOCHS)
        num_warmup_steps = int(num_train_steps / 10)
        scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)

        uniqe_exp_name = f"{config.EXP_NAME}_f{fold}"
        wandb.init(project='Ventilator', entity='trtd56', name=uniqe_exp_name, group=config.EXP_NAME)
        wandb_config = wandb.config
        wandb_config.fold = fold
        for k, v in dict(vars(config)).items():
            if k[:2] == "__" or k in config.NOT_WATCH_PARAM:
                continue
            wandb_config[k] = v
        wandb.watch(model)
        
        os.makedirs(f'{config.OUTPUT}/{config.EXP_NAME}', exist_ok=True)
        model_path = f"{config.OUTPUT}/{config.EXP_NAME}/ventilator_f{fold}_best_model.bin"
        
        valid_best_score = float('inf')
        valid_best_score_mask = float('inf')
        for epoch in tqdm(range(config.N_EPOCHS)):
            train_loss, lrs = train_loop(model, optimizer, scheduler, train_loader)
            valid_loss, valid_predict = valid_loop(model, valid_loader, target_dic_inv)
            valid_score = np.abs(valid_predict - train_df.query(f"fold=={fold}")['pressure'].values).mean()

            mask = (train_df.query(f"fold=={fold}")['u_out'] == -1).values  # u_out is normalized [-1, 0]
            _score = valid_predict - train_df.query(f"fold=={fold}")['pressure'].values
            valid_score_mask = np.abs(_score[mask]).mean()

            if valid_score < valid_best_score:
                valid_best_score = valid_score
                torch.save(model.state_dict(), model_path)
                oof[train_df.query(f"fold=={fold}").index.values] = valid_predict

            if valid_score_mask < valid_best_score_mask:
                valid_best_score_mask = valid_score_mask

            wandb.log({
                "train_loss": train_loss,
                "valid_loss": valid_loss,
                "valid_score": valid_score,
                "valid_best_score": valid_best_score,
                "valid_score_mask": valid_score_mask,
                "valid_best_score_mask": valid_best_score_mask,
                "learning_rate": lrs,
            })
            
            torch.cuda.empty_cache()
            gc.collect()
        
        model.load_state_dict(torch.load(model_path))
        test_preds = test_loop(model, test_loader, target_dic_inv)
        test_preds_lst.append(test_preds)
        
        sub_df['pressure'] = test_preds
        sub_df.to_csv(f"{config.OUTPUT}/{config.EXP_NAME}/sub_f{fold}.csv", index=None)

        train_df['oof'] = oof
        train_df.to_csv(f"{config.OUTPUT}/{config.EXP_NAME}/oof.csv", index=None)
        wandb.finish()

        del model, optimizer, scheduler, train_loader, valid_loader, train_dset, valid_dset
        torch.cuda.empty_cache()
        gc.collect()
    
    sub_df['pressure'] = np.stack(test_preds_lst).mean(0)
    sub_df.to_csv(f"{config.OUTPUT}/{config.EXP_NAME}/submission_mean.csv", index=None)

    sub_df['pressure'] = np.median(np.stack(test_preds_lst), axis=0)
    sub_df.to_csv(f"{config.OUTPUT}/{config.EXP_NAME}/submission_median.csv", index=None)
    
    # Post Processing: https://www.kaggle.com/snnclsr/a-dummy-approach-to-improve-your-score-postprocess
    unique_pressures = train_df["pressure"].unique()
    sorted_pressures = np.sort(unique_pressures)
    total_pressures_len = len(sorted_pressures)

    def find_nearest(prediction):
        insert_idx = np.searchsorted(sorted_pressures, prediction)
        if insert_idx == total_pressures_len:
            # If the predicted value is bigger than the highest pressure in the train dataset,
            # return the max value.
            return sorted_pressures[-1]
        elif insert_idx == 0:
            # Same control but for the lower bound.
            return sorted_pressures[0]
        lower_val = sorted_pressures[insert_idx - 1]
        upper_val = sorted_pressures[insert_idx]
        return lower_val if abs(lower_val - prediction) < abs(upper_val - prediction) else upper_val
    
    sub_df = pd.read_csv(f"{config.OUTPUT}/{config.EXP_NAME}/submission_mean.csv")
    sub_df["pressure"] = sub_df["pressure"].apply(find_nearest)
    sub_df.to_csv(f"{config.OUTPUT}/{config.EXP_NAME}/submission_mean_pp.csv", index=None)
    
    sub_df = pd.read_csv(f"{config.OUTPUT}/{config.EXP_NAME}/submission_median.csv")
    sub_df["pressure"] = sub_df["pressure"].apply(find_nearest)
    sub_df.to_csv(f"{config.OUTPUT}/{config.EXP_NAME}/submission_median_pp.csv", index=None)
    
    cv_score = train_df.apply(lambda x: abs(x['oof'] - x['pressure']), axis=1).mean()
    print("CV:", cv_score)

In [ ]:
if __name__ == "__main__":
    main()